In [312]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
import warnings
warnings.filterwarnings('ignore')

In [313]:
table_data = pd.read_csv('data_table.csv', index_col='Сырье')
prices = pd.read_csv('prices.csv')

In [314]:
table_data.head()

,Запасы сырья,Затраты на стол,Затраты на стул,Затраты на бюро,Затраты на книжный шкаф
Сырье,,,,,
Доски 1,1500,5,1,9,12
Доски 2,1000,2,3,4,13
Труд. затраты,800,3,2,5,10


In [315]:
prices.head()

,Цена стола,Цена стула,Цена бюро,Цена книжного шкафа
0,12,5,15,10


$\Large F=c_{1}*x_{1}+c_{2}*x_{2}+c_{3}*x_{3}+c_{4}*x_{4} \xrightarrow{} max $

$\Large \begin{cases} a_{11}*x_1 + a_{12}*x_2 + a_{13}*x_3 + a_{14}*x_4 \leq b_1 \\ a_{21}*x_1 + a_{22}*x_2 + a_{23}*x_3 + a_{24}*x_4 \leq b_2 \\ a_{31}*x_1 + a_{32}*x_2 + a_{33}*x_3 + a_{34}*x_4 \leq b_3 \end{cases}$

In [316]:
def fill_c(prices: pd.DataFrame):
    """
    Считываем переменные целевой функции, умножив их на -1, и заполняем ими массив
    :param: prices: DataFrame with values
    :return: 1D array of values
    """
    return np.negative(prices.values[0])

In [317]:
def fill_a_ub(table_data: pd.DataFrame):
    """
    Считываем переменные левой части уравнения и записываем их в массив
    :param: table_data: DataFrame with values
    :return: 2D array of values
    """
    out_of_materials_stocks = table_data.drop('Запасы сырья', axis=1)
    return out_of_materials_stocks.values

In [318]:
def fill_b_ub(table_data: pd.DataFrame):
    """
    Считываем переменные правой части уравнения и записываем их в массив
    :param: table_data: DataFrame with values
    :return: 1D array of values
    """
    return table_data['Запасы сырья'].values

In [319]:
c = fill_c(prices)
a_ub = fill_a_ub(table_data)
b_ub = fill_b_ub(table_data)

In [320]:
c

array([-12,  -5, -15, -10])

In [321]:
a_ub

array([[ 5,  1,  9, 12],
       [ 2,  3,  4, 13],
       [ 3,  2,  5, 10]])

In [322]:
b_ub

array([1500, 1000,  800])

In [323]:
n = len(b_ub)

$\Large F=12*x_{1}+5*x_{2}+15*x_{3}+10*x_{4} \xrightarrow{} max $

$\Large \begin{cases} 5*x_1 + 1*x_2 + 9*x_3 + 12*x_4 \leq 1500 \\ 2*x_1 + 3*x_2 + 4*x_3 + 13*x_4 \leq 1000 \\ 3*x_1 + 2*x_2 + 5*x_3 + 10*x_4 \leq 800 \end{cases}$

In [324]:
result = linprog(c=c, A_ub=a_ub, b_ub=b_ub, method='revised simplex')

In [325]:
result

     con: array([], dtype=float64)
     fun: -3200.0
 message: 'Optimization terminated successfully.'
     nit: 2
   slack: array([166.66666667, 466.66666667,   0.        ])
  status: 0
 success: True
       x: array([266.66666667,   0.        ,   0.        ,   0.        ])

In [326]:
result.slack

array([166.66666667, 466.66666667,   0.        ])

Анализ на чувствительность - изменение коэффициентов правой части ограничений  
Будем уменьшать запасы

In [327]:
A=a_ub

Создадим матрицу дополнительных векторов размера [n, n], потому что у нас есть n ограничений, которые нужны нам n раз

In [328]:
A_slack = np.ones([n, n])

Получим номера базисных векторов

In [329]:
# основные вектора
base_ind = np.nonzero(result.x)[0]
# доп вектора
base_ind_extra = np.nonzero(result.slack)[0]+n
base_ind, base_ind_extra

(array([0]), array([3, 4]))

Объединим два списка базисных векторов

In [330]:
base_ind = np.concatenate((base_ind, base_ind_extra))
base_ind

array([0, 3, 4])

Формируем базисную матрицу и столбец базисных коэффициентов целевой функции

In [331]:
basis = []
c_bas = []

for i in range(n):
    if base_ind[i] < n:
        basis.append(A[:, base_ind[i]])
        c_bas.append(c[base_ind[i]])
    else:
        basis.append(A_slack[:, base_ind[i]-n])
        c_bas.append(0)
        
basis = np.reshape(basis, (n, n)).T

Получим обратную матрицу

In [332]:
basis_inv = np.linalg.inv(basis)

#### Запустим пошаговый анализ:
1. Для этого получим вектор $cb*B^{-1}$
2. Получим оценки основных векторов
3. Получим оценки дополнительных векторов
4. Получим коэффициенты разложения вектора $P_0$
5. Проверяем знак коэффициентов разложения вектора $P_0$
6. Проверим не есть ли наше решение оптималным, если так, прекращаем работу цикла. Иначе:
7. Запоминаем индект отрицального коэффициента
8. Находим вектор, который выводится из базиса
9. Вычисляем коэффициенты разложения по базису вектора, который вводится в базис
10. Находим вектор, который выводится из базиса
11. Делаем пересчёт по формулам базиса, а именно:
    * Заменяем номер базисного вектора
    * Заменяем коэффициент целевой функции при базисном векторе 

In [355]:
def sensitivity_analysis(b_ub_new :list):
    """
    Основная функция анализа на чувствительность.
    :params:b_ub_new: 1D array of new values on the right side of the constraint
    :return:
            nit: number of iterations
            p_0: expansion coefficients of the vector P_0
            base_ind: coefficients of basis vector
            fx: new value of objective function
    """
    nit = 0
    B = np.array(b_ub_new)
    while True:
        cb = np.dot(c_bas, basis_inv)
        delta_main = np.dot(cb, A) - c
        delta_extra = np.dot(cb, A_slack)
        delta = np.concatenate((delta_main, delta_extra))
        p_0 = np.dot(basis_inv, B)

        if np.min(p_0) >= 0:
            print('Решение достигло оптимума')
            break
        else:
            ind = np.argmin(p_0)

        index = -1
        minimum = 10000
        for i in range(n):
            find_min = np.dot(basis_inv[ind, :], A_slack[:, i]) 
            if find_min < 0:
                if abs(delta[i]/find_min) < minimum:
                    index = i
                    minimum = delta[i]/find_min

        if index < n:
            p_j = np.dot(basis_inv, A[:, index])
            c_new_basis = c[index]
        else:
            p_j = np.dot(basis_inv, A[:, index-n])
            c_new_basis = 0


        base_ind[ind] = index
        c_bas[ind] = c_new_basis

        basis_inv[ind, :] /= p_j[ind]
        for i in range(n):
            if i != ind:
                basis_inv[i, :] -= basis_inv[ind, :]*p_j[i]
        nit += 1
        if nit > 100000:
            break
    fx = np.dot(p_0, c_bas)
    return [nit, p_0, base_ind, fx]

In [356]:
def show_result(result):
    """
    Выводим результат наших вычислений.
    :params:result: 1D array of values with returned in sensitivity_analysis
    :return: nothing
    """
    print('Количество интераций: {}\nКоэффициентры разложения вектора p_0: {}' \
          '\nБазисный векор: {}\nНовое значение целевой функции: {}'.format(result[0], result[1], result[2], result[3]))

Уменьшим запасы доски 1 с 1500 до 1300, то есть уменьшим $b_1$ до 1300

In [357]:
show_result(sensitivity_analysis([1300, 1000, 800]))

Решение достигло оптимума
Количество интераций: 0
Коэффициентры разложения вектора p_0: [  0.    0.  112.5]
Базисный векор: [ 0  3 -1]
Новое значение целевой функции: -1124.9999999999995


Уменьшим запасы доски 2 с 1000 до 800, то есть уменьшим $b_2$ до 800

In [345]:
show_result(sensitivity_analysis([1500, 800, 800]))

Решение достигло оптимума
Количество интераций: 0
Коэффициентры разложения вектора p_0: [ 0.   0.  87.5]
Базисный векор: [ 0  3 -1]
Новое значение целевой функции: -875.0


Уменьшим трудовые запасы с 800 до 200, то есть уменьшим $b_3$ до 200

In [369]:
show_result(sensitivity_analysis([1500, 1000, 200]))

Решение достигло оптимума
Количество интераций: 0
Коэффициентры разложения вектора p_0: [  0.    0.  362.5]
Базисный векор: [ 0  3 -1]
Новое значение целевой функции: -3624.999999999999


Попробуем уменьшить запасы досок, исходя из того, что трудовые затраты можно сократить до 200

In [378]:
show_result(sensitivity_analysis([1500, 950, 200]))

Решение достигло оптимума
Количество интераций: 0
Коэффициентры разложения вектора p_0: [  0.   0. 350.]
Базисный векор: [ 0  3 -1]
Новое значение целевой функции: -3499.999999999999


In [379]:
show_result(sensitivity_analysis([1450, 1000, 200]))

Решение достигло оптимума
Количество интераций: 0
Коэффициентры разложения вектора p_0: [  0.     0.   356.25]
Базисный векор: [ 0  3 -1]
Новое значение целевой функции: -3562.499999999999


Делаем вывод, что запасы досок находятся в своём оптимуме: 1500м досок первого типа и 1000м досок второго типа. При этом трудовые запасы на изготовление продукции можно сокращать. Это связано с тем, что сотрудники без дела - так же получают зарплату, значит нужно сократить трудовые запасы, чтобы сотрудники не простаивались по истечению ресурсов досок